### Data Ingestion Module

In [1]:
import os

%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/research'

In [2]:
os.chdir("../")
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer'

### Basic Configuration

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path


In [4]:
from src.text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.text_summarizer.utils.common import create_directories, read_yaml


## Configuration Updates 

In [5]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_path)
        self.paramss = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config


In [6]:
import os
import zipfile
from urllib import request

from src.text_summarizer.logging import logger


### Components

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def downlaod_file(self) -> None:
        if not Path(self.config.local_data_file).exists():
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, filename=self.config.local_data_file
            )
            logger.info("File is downloaded")
        else:
            logger.info("File already exits")

    def extract_zip_file(self) -> None:
        """
        Extracts the zip file into the data directory.

        Args:
            Zip_file_path: str (path to zip file)
        Function returns None.

        """
        unzip_path = self.config.unzip_dir
        Path(unzip_path).mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.downlaod_file()
data_ingestion.extract_zip_file()

[2025-06-24 22:46:37,682: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-24 22:46:37,683: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-24 22:46:37,684: INFO: common: created directory at: artifacts]
[2025-06-24 22:46:37,685: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-24 22:46:38,429: INFO: 1397443728: File is downloaded]
